# SVM

In [8]:
import numpy as np
import pandas as pd
import os
from libsvm.svmutil import svm_train, svm_predict, svm_problem
import itertools
from scipy.spatial.distance import cdist

## Part 1.
Use SVM models to tackle classification on images of hand-written digits.

In [3]:
def load_data(data_path):
    X_train = pd.read_csv(os.path.join(data_path, "X_train.csv"), header=None).to_numpy()
    y_train = pd.read_csv(os.path.join(data_path, "Y_train.csv"), header=None).to_numpy().reshape(-1)
    X_test = pd.read_csv(os.path.join(data_path, "X_test.csv"), header=None).to_numpy()
    y_test = pd.read_csv(os.path.join(data_path, "Y_test.csv"), header=None).to_numpy().reshape(-1)

    return X_train, y_train, X_test, y_test

In [4]:
X_train, y_train, X_test, y_test = load_data("./data")

### svm_train usage
<div align="center">
<img src="./img/svm_train_usage.png" width = "600" alt="svm_train usage" align=center />
</div>

In [5]:
kernel_type = {
    "linear": 0,
    "polynomial": 1,
    "RBF": 2}

In [5]:
for key, value in kernel_type.items():
    m = svm_train(y_train, X_train, f"-q -t {value}")
    p_labels, p_acc, p_vals = svm_predict(y_test, X_test, m, "-q")
    print(f"kernel_type: {key}\taccuracy: {p_acc[0]:.2f}")

kernel_type: linear	accuracy: 95.08
kernel_type: polynomial	accuracy: 34.68
kernel_type: RBF	accuracy: 95.32


## Part 2
Please use C-SVC. Since there are some parameters you need to tune for, please do the grid search for finding parameters of the best performing model. For instance, in C-SVC you have a parameter C, and if you use RBF kernel you have another parameter 𝛾, you can search for a set of (C, 𝛾) which gives you best performance in cross-validation.

In [23]:
def gridSearch(X_train, y_train, **param):
    kernel_type = param.get("kernel_type", 0)
    C = param.get("C", [1])
    gamma = param.get("gamma", [1 / X_train.shape[1]])
    coef0 = param.get("coef0", [0])
    degree = param.get("degree", [3])

    combinations = [C, gamma, coef0, degree]
    best_acc = 0
    best_comb = None
    for comb in list(itertools.product(*combinations)):
        acc = svm_train(y_train, X_train, f"-q -t {kernel_type} -v 3 -c {comb[0]} -g {comb[1]} -r {comb[2]} -d {comb[3]}")
        if acc > best_acc:
            best_acc = acc
            best_comb = comb

    print(f"best combination (C, gamma, coef0, degree): {best_comb}\tbest accuracy: {best_acc}")
    return best_comb, best_acc

### Linear kernel

In [26]:
param = {"kernel_type": kernel_type['linear'], "C": [10**x for x in range(-5, 6)]}
best_comb, best_acc = gridSearch(X_train, y_train, **param)

Cross Validation Accuracy = 79.48%
Cross Validation Accuracy = 88.32%
Cross Validation Accuracy = 95.3%
Cross Validation Accuracy = 96.92%
Cross Validation Accuracy = 96.88%
Cross Validation Accuracy = 96.2%
Cross Validation Accuracy = 96.2%
Cross Validation Accuracy = 96.3%
Cross Validation Accuracy = 96.46%
Cross Validation Accuracy = 96.48%
Cross Validation Accuracy = 96.48%
best combination (C, gamma, coef0, degree): (0.01, 0.0012755102040816326, 0, 3)	best accuracy: 96.92


In [27]:
m = svm_train(y_train, X_train, f"-q -t 0 -c {best_comb[0]}")
p_labels, p_acc, p_vals = svm_predict(y_test, X_test, m, "-q")
print(f"linear kernel after grid search accuracy: {p_acc[0]:.2f}")

linear kernel after grid search accuracy: 95.96


### Polynomial kernel

In [29]:
param = {"kernel_type": kernel_type['polynomial'], 
         "C": [10**x for x in range(-3, 4)],
         "gamma": [10**x for x in range(-3, 4)],
         "coef0": [x for x in range(-1, 2)],
         "degree": [x for x in range(2, 5)]}
best_comb, best_acc = gridSearch(X_train, y_train, **param)

Cross Validation Accuracy = 0.18%
Cross Validation Accuracy = 81.9%
Cross Validation Accuracy = 0.26%
Cross Validation Accuracy = 45.84%
Cross Validation Accuracy = 28.46%
Cross Validation Accuracy = 23.76%
Cross Validation Accuracy = 78.38%
Cross Validation Accuracy = 76.68%
Cross Validation Accuracy = 75.72%
Cross Validation Accuracy = 0.26%
Cross Validation Accuracy = 84.72%
Cross Validation Accuracy = 0.6%
Cross Validation Accuracy = 46.2%
Cross Validation Accuracy = 29.32%
Cross Validation Accuracy = 23.7%
Cross Validation Accuracy = 69.92%
Cross Validation Accuracy = 76.04%
Cross Validation Accuracy = 83.5%
Cross Validation Accuracy = 91.16%
Cross Validation Accuracy = 92.3%
Cross Validation Accuracy = 91.58%
Cross Validation Accuracy = 94.32%
Cross Validation Accuracy = 96.12%
Cross Validation Accuracy = 96.12%
Cross Validation Accuracy = 95.12%
Cross Validation Accuracy = 97.3%
Cross Validation Accuracy = 97.38%
Cross Validation Accuracy = 98.08%
Cross Validation Accuracy = 97.

In [30]:
m = svm_train(y_train, X_train, f"-q -t 1 -c {best_comb[0]} -g {best_comb[1]} -r {best_comb[2]} -d {best_comb[3]}")
p_labels, p_acc, p_vals = svm_predict(y_test, X_test, m, "-q")
print(f"polynomial kernel after grid search accuracy: {p_acc[0]:.2f}")

polynomial kernel after grid search accuracy: 97.80


### Radial basis kernel

In [31]:
param = {"kernel_type": kernel_type['RBF'], 
         "C": [10**x for x in range(-3, 4)],
         "gamma": [10**x for x in range(-3, 4)]}
best_comb, best_acc = gridSearch(X_train, y_train, **param)

Cross Validation Accuracy = 81.18%
Cross Validation Accuracy = 89.68%
Cross Validation Accuracy = 49.92%
Cross Validation Accuracy = 20.56%
Cross Validation Accuracy = 78.88%
Cross Validation Accuracy = 35.92%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 81%
Cross Validation Accuracy = 91.74%
Cross Validation Accuracy = 51.18%
Cross Validation Accuracy = 20.54%
Cross Validation Accuracy = 78.98%
Cross Validation Accuracy = 35.72%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 91.8%
Cross Validation Accuracy = 96.14%
Cross Validation Accuracy = 54.04%
Cross Validation Accuracy = 20.56%
Cross Validation Accuracy = 78.7%
Cross Validation Accuracy = 36.26%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 96.1%
Cross Validation Accuracy = 97.64%
Cross Validation Accuracy = 91.26%
Cross Validation Accuracy = 29.42%
Cross Validation Accuracy = 26.94%
Cross Validation Accuracy = 36%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 97.12%
Cross 

In [32]:
m = svm_train(y_train, X_train, f"-q -t 2 -g {best_comb[1]}")
p_labels, p_acc, p_vals = svm_predict(y_test, X_test, m, "-q")
print(f"radial basis kernel after grid search accuracy: {p_acc[0]:.2f}")

radial basis kernel after grid search accuracy: 97.52


## Part 3.
Use linear kernel + RBF kernel together (therefore a new kernel function) and compare its performance with respect to others.

### Precomputed kernel usage
<div align="center">
<img src="./img/precomputed_kernel_usage.png" width = "400" alt="precomputed kernel usage" align=center />
</div>

In [10]:
def linearRBF(X, X_, gamma):
    linear = X @ X_.T
    RBF = np.exp(-gamma * cdist(X, X_, 'sqeuclidean'))
    kernel = linear + RBF
    kernel = np.hstack((np.arange(1, len(X)+1).reshape(-1, 1), kernel))

    return kernel

In [14]:
best_comb = [10, 0.01]

In [15]:
K = linearRBF(X_train, X_train, best_comb[1])
KK = linearRBF(X_test, X_train, best_comb[1])

In [16]:
prob = svm_problem(y_train, K, isKernel=True)
m = svm_train(prob, f"-q -t 4 -c {best_comb[0]}")
p_labels, p_acc, p_vals = svm_predict(y_test, KK, m, "-q")
print(f"kernel type: linear + RBF kernel\taccuracy: {p_acc[0]:.2f}")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()